Working on adding a lot of features just to see if it can get the score up regardless of how complicated or where the data is coming from

Features that I will be adding

* Taget given surface counts
* words from the target and source document
  * possible back prop into these vectors, idea is to replace tf-idf with some nn and back prop here
* using a linear layer near the output to combine the features

* adding the indicator features from the berkeley entity system
  * I guess that these will be on the final linear layer
  
* the tesor product type stuff did not really work, going to try and maybe increase the dim of the convs somehow

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer, WordTokenizer
import json
import re
import random

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki-aida1.json') as f:
    externalWikiJ = json.load(f)
    queries = externalWikiJ['queries']
    featuresNames = externalWikiJ['featIndex']

In [4]:
len(featuresNames)

10415

In [5]:
len(queries)

1146

In [6]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

1146

In [7]:
sum(sum(not g['training'] for g in v.values()) for v in queries.values())

3805

In [8]:
sum(sum(g['training'] for g in v.values()) for v in queries.values())

14649

In [9]:
len(featuresNames)

10415

In [10]:
max(max(max(max(a[1]) for a in g['vals'].values()) for g in v.values()) for v in queries.values())

10353

In [11]:
# for q,v in queries.items():
#     if not v.values()[0]['training']:
#         del queries[q]

In [12]:
# # load the samples from the provided testing set

# with open('/data/matthew/external-wiki-testing.json') as f:
#     queries.update(**json.load(f)['queries'])

In [13]:
for q,v in queries.items():
    for v2 in v.values():
        v2['gold'] = v2['gold'].replace('_', ' ')

In [14]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links7-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirect7.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [15]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [16]:
len(wordvectors.vectors)

4056055

In [17]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-surface-counts7.json') as f:
    surface_counts = json.load(f)

In [18]:
# try and make the surfaces items match what we are looking for
surface_counts_re = re.compile('([\.,!\?])')
for sk in surface_counts.keys():
    nsk = sk.replace('(', '-lrb-').replace(')', '-rrb-')
    nsk = surface_counts_re.sub(' \\1', nsk)
    if nsk != sk:
        surface_counts[nsk] = surface_counts[sk]

In [19]:
from wikireader import WikiRegexes, WikipediaReader

In [20]:
def PreProcessedQueries(wikipedia_dump_fname, vectors=wordvectors, queries=queries, redirects=page_redirects, surface=surface_counts):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    wordvec = WordTokenizer(vectors, sentence_length=200)
    documentvec = WordTokenizer(vectors, sentence_length=1)
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                documentvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            documentvec.get_location(redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages
    
    for w in queried_pages:
        wordvec.tokenize(get_words.sub(' ', w))

    page_content = {}

    class GetWikipediaWords(WikipediaReader, WikiRegexes):

        def readPage(ss, title, content, namespace):
            if namespace != 0:
                return
            tt = ss.convertToTitle(title)
            if tt in queried_pages:
                cnt = ss._wikiToText(content)
                page_content[tt] = wordvec.tokenize(cnt)

    GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content
    rs = surface

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        documentvecs = documentvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        surface_counts = rs
        
        
    return PreProcessedQueriesCls

In [21]:
%time basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

CPU times: user 6min 44s, sys: 14.3 s, total: 6min 58s
Wall time: 7min 5s


In [22]:
len(wordvectors.vectors)

4056055

In [23]:
len(basePreProcessedQueries.wordvecs.reverse_word_location)

168419

In [24]:
for qu in queries.values():
    for en in qu.values():
        en['boosted'] = 1.0

In [25]:
def cleanUpMultipleLinks():
    for qu in queries.values():
        for en in qu.values():
            gold_page = en['gold']
            gold_title = WikiRegexes.convertToTitle(gold_page)
            gold_title = page_redirects.get(gold_title, gold_title)
            pages = set()
            for p in en['vals'].keys():
                wiki_title = WikiRegexes.convertToTitle(p)
                wiki_title = page_redirects.get(wiki_title, wiki_title)
                if wiki_title == gold_title and p != en['gold']:
                    del en['vals'][p]
                elif wiki_title not in pages:
                    pages.add(wiki_title)
                else:
                    del en['vals'][p]
cleanUpMultipleLinks()

In [26]:
sum(len(q) for q in queries)

1357920

In [27]:
# def removeSingleLinkTargets():
#     "items that the surface link set only has a single item are trival, so remove"
#     get_link = re.compile('.*?\[(.*?)\].*?')
#     for quk, qu in queries.items():
#         for sur in qu.keys():
#             surlink = get_link.match(sur).group(1)
#             surmatch = surlink.lower()
#             surcounts = surface_counts.get(surmatch)
#             if surcounts and len(surcounts) == 1:
#                 # this is trival since there is only one item
#                 del qu[sur]
#         if not qu:
#             # we removed all the links on this page, remove it otherwise the program crashes
#             del queries[quk]
# removeSingleLinkTargets()

In [28]:
queries.values()[0].values()[0]['vals'].values()[0]

[0, [35]]

In [56]:
class EntityVectorLinkExp(basePreProcessedQueries):

    batch_size = 250 #20000
    num_training_items = 500000 #200000
    dim_compared_vec = 100  # 100

    def __init__(self):
        self.sentence_length = self.wordvecs.sentence_length
        self.sentence_length_short = 10
        self.document_length = 100
        
        self.num_words_to_use_conv = 5
        self.enable_boosting = False
        self.num_negative_target_samples = 1
        #self.enable_match_surface = False
        #self.enable_link_counts = True
        self.enable_train_wordvecs = False
        self.enable_cap_boosting = True
        
        self.num_indicator_features = len(featuresNames)
        
        self.main_nl = lasagne.nonlinearities.leaky_rectify

        self._setup()

    def _setup(self):
        self.x_document_input = T.imatrix('x_doc')  # words from the source document

        self.x_document_id = T.ivector('x_doc_id')  # index of which source doucment this is from
        self.x_surface_text_input = T.imatrix('x_surface_link')  # text of the surface link
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  #  words surrounding the surface link

        self.x_target_input = T.ivector('x_target')  # id of the target vector
        self.x_target_words = T.imatrix('x_target_words')  # words from the target title link
        self.x_matches_surface = T.ivector('x_match_surface')  # indicator if the target title matches the surface
        self.x_matches_counts = T.imatrix('x_matches_counts')  # info about the link counts
        self.x_target_document_words = T.imatrix('x_target_document_words')  # words from the body of target document
        self.x_link_id = T.ivector('x_link_id')  # indx of what link to compare to in the matrix

        self.x_indicator_features = T.matrix('x_indicator_features', dtype='int8')
        
        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # (Not used) contains the location of the gold answer so we can compute the loss
        self.y_grouping = T.imatrix('y_grouping')  # matrix containing [start_idx, end_idx, gold_idx]
        self.y_boosted = T.vector('y_boosted')  # only used if boosting enabled, vector of how much to boost items

        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX),name='embedding_W')
        self.embedding_W_docs = theano.shared(self.documentvecs.get_numpy_matrix().astype(theano.config.floatX),name='embedding_W_docs')
        
        self.document_l = lasagne.layers.InputLayer(
            (None,self.document_length),
            input_var=self.x_document_input
        )

        self.document_embedding_l = EmbeddingLayer(
            self.document_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )
        
        self.document_simple_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=self.dim_compared_vec,
            filter_size=(2, self.wordvecs.vector_size),
            name='document_simple_conv',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
        self.document_simple_sum_l = lasagne.layers.Pool2DLayer(
            #lasagne.layers.reshape(self.document_embedding_l, ([0],[3],[2],1)),
            self.document_simple_conv1_l,
            name='document_simple_pool',
            pool_size=(self.document_length - 2, 1),
            mode='sum',
        )
        
        self.document_conv1_l = lasagne.layers.Conv2DLayer(
            self.document_embedding_l,
            num_filters=30,  # was 75, 100, 500
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='document_conv1',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )

        self.document_max_l = lasagne.layers.Pool2DLayer(
            self.document_conv1_l,
            name='document_pool1',
            pool_size=(self.document_length - self.num_words_to_use_conv, 1),
            mode='max',  # was sum
        )

        document_output_length = 25 # was 100, 200
        
        self.document_dens1 = lasagne.layers.DenseLayer(
            self.document_max_l,
            num_units=document_output_length,
            name='doucment_dens1',
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )

#         self.document_drop1 = lasagne.layers.DropoutLayer(
#             self.document_dens1,
#             p=.25,
#         )

#         self.document_dens2 = lasagne.layers.DenseLayer(
#             self.document_drop1,
#             num_units=225,
#             name='document_dens2',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop2 = lasagne.layers.DropoutLayer(
#             self.document_dens2,
#             p=.25,
#         )

#         self.document_dens3 = lasagne.layers.DenseLayer(
#             self.document_drop2,
#             num_units=document_output_length,
#             name='document_dens3',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

        self.document_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.document_simple_sum_l, ([0],-1)))
    
    
        ##########################################
        ## surface text

        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )

        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=self.dim_compared_vec,  # was 300
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=self.main_nl,
        )

#         self.surface_context_avg1_l = SimpleAverageLayer(
#             [self.surface_context_conv1_l, self.surface_context_l],
#             #name='surface_context_avg'
#         )

        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )
    
        self.surface_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.surface_context_pool1_l, ([0], -1))
        )

        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length_short),
            input_var=self.x_surface_text_input
        )

        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=self.dim_compared_vec,  # was 300
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=self.main_nl,
        )

#         self.surface_avg1_l = SimpleAverageLayer(
#             [self.surface_conv1_l, self.surface_input_l],
#             #name='surface_avg'
#         )

        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length_short - self.num_words_to_use_conv, 1),
            mode='sum',
        )
    
        self.surface_words_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.surface_pool1_l, ([0], -1))
        )

#         self.surface_merged_l = lasagne.layers.ConcatLayer(
#             [self.surface_context_pool1_l, self.surface_pool1_l]
#         )

#         self.surface_dens1 = lasagne.layers.DenseLayer(
#             self.surface_merged_l,
#             name='surface_dens1',
#             num_units=200,  # was 250
#             nonlinearity=self.main_nl,# lasagne.nonlinearities.tanh,
#         )

# #         self.surface_drop1 = lasagne.layers.DropoutLayer(
# #             self.surface_dens1,
# #             p=.25,
# #         )

# #         self.surface_dens2 = lasagne.layers.DenseLayer(
# #             self.surface_drop1,
# #             name='surface_dens2',
# #             num_units=200,
# #             nonlinearity=lasagne.nonlinearities.tanh,
# #         )

#         ##################################################
#         ## merge the documents with the surface info

#         self.document_aligned_l = InputLayer(
#             (None, document_output_length),
#             input_var=self.document_output[self.x_document_id,:]
#         )

#         self.source_l = lasagne.layers.ConcatLayer(
#             [self.document_aligned_l, self.surface_dens1]
#         )

#         self.source_dens1 = lasagne.layers.DenseLayer(
#             self.source_l,
#             num_units=300,
#             name='source_dens1',
#             nonlinearity=self.main_nl,# lasagne.nonlinearities.tanh,
#         )

#         self.source_drop1 = lasagne.layers.DropoutLayer(
#             self.source_dens1,
#             p=.25,
#         )

#         self.source_dens12 = lasagne.layers.DenseLayer(
#             self.source_drop1,
#             num_units=250,
#             name='source_dens12',
#             nonlinearity=self.main_nl,# lasagne.nonlinearities.tanh,
#         )

#         self.source_drop12 = lasagne.layers.DropoutLayer(
#             self.source_dens12,
#             p=.25,
#         )

#         compared_vector_size = 150 #self.wordvecs.vector_size #+ 2 # extra space for if it matches the surface text

#         self.source_dens2 = lasagne.layers.DenseLayer(
#             self.source_drop12,
#             num_units=compared_vector_size,  # this is the same size as the learned wikipedia vectors
#             name='source_dens2',
#             nonlinearity=lasagne.nonlinearities.linear,
#         )

#         self.source_out = lasagne.layers.get_output(self.source_dens2)
        
        
        ###################################################
        ## dealing with the target side

        matched_surface_reshaped = self.x_matches_surface.reshape(
            (self.x_matches_surface.shape[0], 1, 1, 1)).astype(theano.config.floatX)

        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )
        
        #################################
        ## target indicators features

        self.target_matched_surface_input_l = lasagne.layers.InputLayer(
            (None,1,1,1),
            input_var=matched_surface_reshaped,
        )
        
        self.target_matched_counts_input_l = lasagne.layers.InputLayer(
            (None,5),
            input_var=self.x_matches_counts.astype(theano.config.floatX),
        )

        self.target_embedding_l = EmbeddingLayer(
            lasagne.layers.reshape(self.target_input_l, ([0], 1)),
            W=self.embedding_W_docs,
            add_word_params=False,
        )

#         self.target_combined_feats_l = lasagne.layers.ConcatLayer(
#             [self.target_embedding_l, self.target_matched_surface_input_l,
#             lasagne.layers.reshape(self.target_matched_counts_input_l, ([0],1,1,[1]))],
#             axis=3
#         )

        # words from the title of the target
        self.target_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length_short),
            input_var=self.x_target_words,
        )

        self.target_words_embedding_l = EmbeddingLayer(
            self.target_words_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_words_embedding_l,
            name='target_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=self.dim_compared_vec,  # was 75, 150, 350
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )

        self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_words_conv1_l,
            name='target_wrds_pool1',
            pool_size=(self.sentence_length_short - self.num_words_to_use_conv, 1),
            mode='sum',  # was sum
        )
        
        self.target_title_out = lasagne.layers.get_output(
            lasagne.layers.reshape(self.target_words_pool1_l, ([0],-1))
        )
        
        
        # words from the body of the target
        self.target_body_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length),
            input_var=self.x_target_document_words,
        )
        
        self.target_body_words_embedding_l = EmbeddingLayer(
            self.target_body_words_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )
        
        self.target_body_simple_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_body_words_embedding_l,
            name='target_body_simple_conv',
            filter_size=(2, self.wordvecs.vector_size),
            num_filters=self.dim_compared_vec,
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
        self.target_body_simple_sum_l = lasagne.layers.Pool2DLayer(
            #lasagne.layers.reshape(self.target_body_words_embedding_l, ([0],[3],[2],1)),
            self.target_body_simple_conv1_l,
            name='target_body_simple_sum',
            pool_size=(self.sentence_length - 2, 1),
            mode='sum',
        )
        
        self.target_body_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_body_words_embedding_l,
            name='target_body_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=150,
            nonlinearity=lasagne.nonlinearities.leaky_rectify,
        )
        
        self.target_body_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_body_words_conv1_l,
            name='target_body_wrds_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='max',
        )
    
        self.target_merge_l = lasagne.layers.ConcatLayer(
            [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
             lasagne.layers.reshape(self.target_body_words_pool1_l, ([0], [1])),
            # lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))
            ]
        )

#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_merge_l,
#             name='target_wrds_dens1',
#             num_units=400,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )

#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_wrds_dens1',
#             num_units=compared_vector_size,
#             nonlinearity=lasagne.nonlinearities.linear,
#         )

#         self.target_simple = lasagne.layers.DenseLayer(
#             self.target_embedding_l, #self.target_combined_feats_l,
#             name='target_simple1',
#             num_units=compared_vector_size,
#             nonlinearity=lasagne.nonlinearities.linear,
#         )
    
#         self.target_simple2 = lasagne.layers.DenseLayer(
#             self.target_merge_l,
#             name='target_simple2',
#             num_units=compared_vector_size,
#             nonlinearity=lasagne.nonlinearities.linear,
#         )

#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_conv1_l,
#             name='target_dens1',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )

#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )



        #self.target_out = lasagne.layers.get_output(self.target_embedding_l)


#         self.target_out = T.concatenate(
#             [self.embedding_W[self.x_target_input],
#              matched_surface_reshaped,
#             1-matched_surface_reshaped],
#              axis=1)


        #self.target_out = self.embedding_W[self.x_target_input]
        #self.target_out = lasagne.layers.get_output(self.target_dens2)

        self.target_out = lasagne.layers.get_output(
            lasagne.layers.reshape(self.target_body_simple_sum_l, ([0],-1)))

        #########################################################
        ## compute the cosine distance between the two layers
       
        # source body
        self.source_aligned_l = self.document_output[self.x_document_id,:][self.x_link_id,:] #self.source_out[self.x_link_id, :]
        # source context
        self.source_context_aligned_l = self.surface_output[self.x_link_id,:]
        # source surface words
        self.source_surface_words_aligned_l = self.surface_words_output[self.x_link_id,:]
        
        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()

        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)

        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
        
        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
        
        def cosinsim(a, b):
            dotted = T.batched_dot(a, b)
            return dotted / (augNorm(a) * augNorm(b) + .001)
        
        
        
        
        
        ##############################################
        ## tensor product stuff
        
        def tensorP(a,b):
            res, _ = theano.scan(
                fn=lambda x_vec, y_vec, x_norm, y_norm: T.concatenate(
                    [x_vec, y_vec, 
                     T.outer(x_vec / x_norm, y_vec / y_norm).flatten()]
                ),
                outputs_info=None,
                sequences=[a,b, augNorm(a), augNorm(b)],
                non_sequences=None
            )
            return res
        
        
        ##################################################
        ## f-life, changing it to dens layers b/c why not
        
#         self.final_merged_l = lasagne.layers.concat(
#             [
#                 self.target_merge_l,  # the conv layer + embedding without any transforms
#                 # results from the layers with the source   
#                 lasagne.layers.InputLayer(
#                     (None,compared_vector_size),
#                     input_var=self.source_aligned_l,
#                 )
#             ]
#         )
        
#         self.final_dens1 = lasagne.layers.DenseLayer(
#             self.final_merged_l,
#             num_units=400,
#             nonlinearity=lasagne.nonlinearities.tanh,
#             name='final_dens1',
#         )
        
#         self.final_drop1 = lasagne.layers.DropoutLayer(
#             self.final_dens1,
#             p=.25,
#         )
        
#         self.final_dens2 = lasagne.layers.DenseLayer(
#             self.final_drop1,
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#             name='final_dens2',
#         )
        
#         self.final_drop2 = lasagne.layers.DropoutLayer(
#             self.final_dens2,
#             p=.25,
#         )
        
#         self.final_dens3 = lasagne.layers.DenseLayer(
#             self.final_drop2,
#             num_units=1,
#             nonlinearity=lasagne.nonlinearities.tanh,
#             name='final_dens3',
#         )
        
#         self.final_output = lasagne.layers.get_output(self.final_dens3)
        
    
    
        ######################################################
        ## indicator feature input
    
        self.indicator_feat_l = lasagne.layers.InputLayer(
            (None, self.num_indicator_features),
            input_var=self.x_indicator_features.astype(theano.config.floatX),
        )
        
        #############################
        ## Linear features combined
        #############################
        
        def comparedVLayers(a,b):
            dv = cosinsim(a,b)
            return lasagne.layers.InputLayer(
                (None,1),
                input_var=dv.reshape((dv.shape[0], 1))
            )
        
        def comparedVLayers2(a,b):
            dv = theano.gradient.disconnected_grad(tensorP(a,b))  # just see how well this performs without learning this
            return lasagne.layers.InputLayer(
                (None,self.dim_compared_vec ** 2 + self.dim_compared_vec*2 ),
                input_var=dv
            )
#             dp1 = lasagne.layers.DropoutLayer(
#                 il,
#                 p=.1,
#             )
#             de1 = lasagne.layers.DenseLayer(
#                 dp1,
#                 num_units=50,
#                 nonlinearity=lasagne.nonlinearities.leaky_rectify,
#                 name='compD1',
#             )
#             dp2 = lasagne.layers.DropoutLayer(
#                 de1,
#                 p=.1,
#             )
#             return lasagne.layers.DenseLayer(
#                 dp2,
#                 num_units=1,
#                 nonlinearity=lasagne.nonlinearities.tanh,
#                 name='linD2',
#             )
            
        
        self.linear_features_combined = lasagne.layers.concat(
            [
#                 lasagne.layers.InputLayer(
#                     (None, 1), 
#                     input_var=self.res_l.reshape((self.res_l.shape[0], 1)),  # changed from res_l
#                 ),
               comparedVLayers(self.target_out, self.source_aligned_l),
               comparedVLayers(self.target_out, self.source_context_aligned_l),
               comparedVLayers(self.target_out, self.source_surface_words_aligned_l),
                
                comparedVLayers(self.target_title_out, self.source_aligned_l),
                comparedVLayers(self.target_title_out, self.source_context_aligned_l),
                comparedVLayers(self.target_title_out, self.source_surface_words_aligned_l),
#                comparedVLayers2(self.target_out, self.source_aligned_l),
#                comparedVLayers2(self.target_out, self.source_context_aligned_l),
#                comparedVLayers2(self.target_out, self.source_surface_words_aligned_l),
            lasagne.layers.reshape(self.target_matched_surface_input_l, ([0],1)),
            self.target_matched_counts_input_l,
            #self.indicator_feat_l
            ],
            axis=1
        )
        
        self.linear_features_dens_l = lasagne.layers.DenseLayer(
            self.linear_features_combined,
            nonlinearity=lasagne.nonlinearities.linear,  # use tanh so that too large of values don't cause the softmax issues
            num_units=1,
            name='linear_final_l',
            W=lasagne.init.Normal(mean=0.0),
        )
        
        self.linear_features_dens_l.W.get_value(borrow=True)[0:6] += 1.0  # set the word vecs positive
#         lin_feat_W = self.linear_features_dens_l.W.get_value(borrow=True)
#         lin_feat_add_one = np.eye(self.dim_compared_vec).reshape(self.dim_compared_vec**2, 1)
#         lin_feat_W[self.dim_compared_vec*2                           :self.dim_compared_vec*2+  self.dim_compared_vec**2] += lin_feat_add_one
#         lin_feat_W[self.dim_compared_vec*4+  self.dim_compared_vec**2:self.dim_compared_vec*4+2*self.dim_compared_vec**2] += lin_feat_add_one
#         lin_feat_W[self.dim_compared_vec*6+2*self.dim_compared_vec**2:self.dim_compared_vec*6+3*self.dim_compared_vec**2] += lin_feat_add_one
        
        self.linear_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.linear_features_dens_l, ([0],))
        )
        
        # rescaled the output so we don't crash the softmax layer with a value that is too large
        # just a hack
        self.linear_output_rescaled = 10 * self.linear_output / theano.gradient.disconnected_grad(abs(self.linear_output).max())
        
        ########################################
        ## true output values
        ########################################
        
        self.true_output = self.linear_output_rescaled
        
        
        
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) *
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))


        #self.golds = self.res_cap[self.y_answer]

#         def maxOverRange(indx):
#             #return T.max(self.res_cap[T.arange(indx[0],indx[1])]) - self.res_cap[indx[2]]
#             #return -( self.res_l[indx[2]] - T.log(T.exp(self.res_l[T.arange(indx[0],indx[1])]).sum()) )
#             return -( self.res_l[indx[2]] - self.res_l[indx[0]])

#         # build a tensor to make a matrix with one set on each dimention
#         self.grouped, grouped_update = theano.scan(maxOverRange, sequences=self.y_grouping)

        def setSubSelector(indx, outputs):
            return T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)

        num_target_samples = self.linear_output.shape[0]

        select_seq = T.concatenate([
            self.y_grouping,
            T.arange(self.y_grouping.shape[0]).reshape((self.y_grouping.shape[0], 1))
        ], axis=1)

        self.selecting_matrix, _ = theano.scan(
            setSubSelector,
            outputs_info=T.zeros((num_target_samples, self.y_grouping.shape[0])), #num_target_samples)),
            #n_steps=self.y_grouping.shape[0]
            sequences=select_seq,
        )

#         self.groupped_elems = T.dot(self.selecting_matrix[-1], 
#                                     T.diag(T.exp(self.true_output)))
#         self.groupped_res = T.log(self.groupped_elems.sum(axis=0)[T.arange(self.y_grouping.shape[0])])
        self.groupped_elems = T.dot(self.selecting_matrix[-1].T, 
                                   T.exp(self.true_output))
        self.groupped_res = T.log(self.groupped_elems)
        
        self.loss_vec = self.groupped_res - self.true_output[self.y_grouping[:,2]]
        
        if self.enable_boosting:
            self.loss_scalar = T.dot(self.y_boosted, self.loss_vec)
        else:
            self.loss_scalar = self.loss_vec.sum()

        self.all_params = (
            #lasagne.layers.get_all_params(self.target_dens2) +
            # TODO: add params for the target stuff,
            #lasagne.layers.get_all_params(self.final_dens3) +
            #lasagne.layers.get_all_params(self.target_simple2) +
            #lasagne.layers.get_all_params(self.source_dens2) +
            lasagne.layers.get_all_params(self.document_simple_sum_l) +
            lasagne.layers.get_all_params(self.surface_context_pool1_l) +
            lasagne.layers.get_all_params(self.surface_pool1_l) +
            lasagne.layers.get_all_params(self.target_body_simple_sum_l) +
            lasagne.layers.get_all_params(self.target_words_pool1_l) + 
            lasagne.layers.get_all_params(self.linear_features_dens_l)
            #lasagne.layers.get_all_params(self.document_dens2)
        )
        
        self.regularization = (self.linear_features_dens_l.W ** 2).sum() / 400

        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))

        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)

        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?

        # this one works reasonably well
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)

        #self.loss_vec = self.grouped

        #self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        #self.loss_vec = T.max(self.res_l[self.y_answer] - self.res_l + .1, 0)

        self.updates = lasagne.updates.adadelta(
            self.loss_scalar / self.loss_vec.shape[0] + self.regularization, 
            self.all_params)

        self.func_inputs = [
            self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, self.x_document_id,
            self.x_target_input, self.x_matches_surface, self.x_matches_counts, self.x_link_id, 
            self.x_target_words, self.x_target_document_words, self.x_indicator_features,
            self.y_answer, self.y_grouping, self.y_boosted
        ]
        
        self.func_outputs = [
            self.true_output,
            self.loss_vec.sum(),
            self.loss_scalar,
            self.loss_vec,
            #self.res_l,
        ]

        ################################################################3
        ## TODO: need to return the actual output layer instead of the res_cap, since that is something else now
        
        self.train_func = theano.function(
            self.func_inputs,
            self.func_outputs,
            updates=self.updates,
            on_unused_input='ignore',
        )

        self.test_func = theano.function(
            self.func_inputs,
            self.func_outputs,
            on_unused_input='ignore',
        )

    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_body_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
        self.current_feat_indicators = []
        self.current_learning_groups = []
        self.learning_targets = []
        self.current_surface_target_counts = []
        self.current_boosted_groups = []
        
        self.failed_match = []
            
    def compute_batch(self, isTraining=True, useTrainingFunc=True):
        if isTraining and useTrainingFunc:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        self.total_boosted_loss = 0.0

        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')

        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] != isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink, length=self.sentence_length_short))
                surmatch = surlink.lower()
                surcounts = self.surface_counts.get(surmatch)
                if not surcounts:
                    self.failed_match.append(surmatch)
                    surcounts = {}
                target_body_words_input = []  # words from the target document
                target_words_input = []  # the words from the target title
                target_matches_surface = []
                target_inputs = []  # the target vector
                target_learings = []
                target_match_counts = []
                target_gold_loc = -1
                target_group_start = len(self.current_target_input)
                target_feat_indicators = []
                
                for target in set(targets['vals'].keys() +
                                 random.sample(self.documentvecs.reverse_word_location, self.num_negative_target_samples)
                                  ) - {None,}:
                    
                    
                    isGold = target == targets['gold']
                    cnt_wrds = self.page_content.get(WikiRegexes.convertToTitle(target))
                    wiki_title = WikiRegexes.convertToTitle(target)
                    cnt = self.documentvecs.get_location(wiki_title)
                    if wiki_title == 'nil':
                        cnt = 0  # this is the stop symbol location
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        continue
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    target_body_words_input.append(cnt_wrds or [0]*self.sentence_length)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target), length=self.sentence_length_short))
                    target_inputs.append(cnt)  
                    # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                    target_match_counts.append(surcounts.get(wiki_title, 0))
                    indicators = np.zeros((self.num_indicator_features,), dtype='int8')
                    indicators_place = targets['vals'].get(target)
                    if indicators_place:
                        indicators[indicators_place[1]] = 1
                    target_feat_indicators.append(indicators)
                    #if wiki_title not in surcounts:
                    #    print surcounts, wiki_title
                if target_gold_loc is not None or not isTraining:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    gold_loc = (len(self.current_target_goal) + target_gold_loc)
                    sorted_match_counts = [-4,-3,-2,-1] + sorted(set(target_match_counts))
                    #print sorted_match_counts
                    target_match_counts_indicators = [
                        [
                            int(s == sorted_match_counts[-1]),
                            int(s == sorted_match_counts[-2]),
                            int(s == sorted_match_counts[-3]),
                            int(0 < s <= sorted_match_counts[-4]),
                            int(s == 0),
                        ]
                        for s in target_match_counts
                    ]
                    self.current_target_goal += [gold_loc] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input
                    self.current_target_matches_surface += target_matches_surface
                    self.current_surface_target_counts += target_match_counts_indicators
                    self.current_target_body_words += target_body_words_input
                    self.current_feat_indicators += target_feat_indicators
                    target_group_end = len(self.current_target_input)
                    self.current_learning_groups.append(
                        [target_group_start, target_group_end,
                         gold_loc])
                    self.current_boosted_groups.append(targets['boosted'])

                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links, self.total_boosted_loss / self.total_links

        if len(self.current_target_id) > 0:
            self.run_batch(func)

        return self.total_loss / self.total_links, self.total_boosted_loss / self.total_links

    def run_batch(self, func):
        res_vec, loss_sum, loss_boosted, loss_vec, = func(
            self.current_documents,
            self.current_surface_link, self.current_surface_context, self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id, 
            self.current_target_words, self.current_target_body_words, self.current_feat_indicators,
            self.current_target_goal, self.current_learning_groups, self.current_boosted_groups,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        self.total_boosted_loss += loss_boosted
        learned_groups = []  # right...dict not hashable....
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            if l[1] in l[0]['vals']:
                l[0]['vals'][ l[1] ][0] = float(res_vec[i]), 0#float(nn_outs[i])
            if l[0] not in learned_groups:
                learned_groups.append(l[0])
        for group in learned_groups:
            if group['gold']:
                correct = max(group['vals']) == group['vals'].get(group['gold'])
                group['boosted'] *= .4 if correct else 2.0
                if self.enable_cap_boosting:
                    if group['boosted'] > 10:
                        group['boosted'] = 10.0
                    elif group['boosted'] < 0.1:
                        group['boosted'] = 0.1
        self.reset_accums()

    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')



queries_exp = EntityVectorLinkExp()

In [57]:
queries_exp.linear_features_dens_l.W.get_value(borrow=True).shape

(12, 1)

In [ ]:
queries_exp.all_params

[document_simple_conv.W,
 document_simple_conv.b,
 surface_cxt_conv1.W,
 surface_cxt_conv1.b,
 surface_conv1.W,
 surface_conv1.b,
 target_body_simple_conv.W,
 target_body_simple_conv.b,
 target_wrds_conv1.W,
 target_wrds_conv1.b,
 linear_final_l.W,
 linear_final_l.b]

In [32]:
def evalCurrentState(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct = 0
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            if en['gold']:
                if all_trained > numSamples:
                    break
                all_measured += 1
                all_trained += len(en['vals'].values())
                m = max(en['vals'].values())
                if en['vals'].get(en['gold']) == m and m != 0:
                    all_correct += 1
           
    r = all_measured, float(all_correct) / all_measured
    print r
    return r

In [33]:
def evalCurrentStateRank(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct_place = 0
    p_counts = dict((k,0) for k in range(0,10))
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            if en['gold']:
                if all_trained > numSamples:
                    break
                svals = sorted(en['vals'].values(), key=lambda x: 0 if not isinstance(x, tuple) else -x[0])
                gv = en['vals'][en['gold']]
                if gv == 0:
                    continue
                all_measured += 1
                for i in xrange(len(svals)):
                    if svals[i] == gv:
                        if i < 10:
                            p_counts[i] += 1
                        all_correct_place += i + 1
                        break

    r = all_measured, float(all_correct_place) / all_measured, p_counts
    print r
    return r


In [34]:
def evalCurrentStateF1(trainingData=True, numSamples=50000):
    correct = 0
    precDenom = 0
    recDenom = 0
    all_trained = 0
    all_measured = 0
    for qu in queries.values():
        if qu.values()[0]['training'] != trainingData:
            continue
        allGold = set()
        allChoosen = set()
        if all_trained > numSamples:
            break
        for en in qu.values():
            if en['gold']:  # we can eval this item
                all_measured += 1
                allGold.add(en['gold'])
                svals = sorted(en['vals'].values())
                picked = None
                for k,v in en['vals'].iteritems():
                    all_trained += 1
                    if v == svals[-1]:
                        picked = k
                allChoosen.add(picked)
                #if svals[0] == svals[1] and en['gold'] != picked:
                #    raise NotImplementedError()
#                 if en['gold'] == picked:
#                     correct += 1
#             if len(svals) > 5 and en['gold'] != picked:
#                 raise NotImplementedError()
        precDenom += len(allChoosen)
        recDenom += len(allGold)
        correct += len(allGold & allChoosen)
    correct = float(correct)
    prec = correct / precDenom
    rec = correct / recDenom
    f1 = 2 * prec * rec / (prec + rec)
    r = all_measured, 'Prec = {}/{} = {}, Rec = {}/{} = {}, F1 = {}'.format(
        correct, precDenom, prec, 
        correct, recDenom, rec, 
        f1)
    print r
    return r

In [35]:
#sorted(queries.values()[0].values()[0]['vals'].values(), key=lambda x: 0 if not isinstance(x, tuple) else -x[0])

In [36]:
# import random
# def augmentTrainingData():
#     for quk in queries.keys():
#         qu = queries[quk]
#         for enk in qu.keys():
#             en = qu[enk]
#             if not en['gold']:
#                 del qu[enk]
#         if not qu:
#             del queries[quk]
#     for qu in queries.values():
#         training = random.random() > .15
#         for en in qu.values():
#             en['training'] = training
# augmentTrainingData()

In [37]:
def findWrongItems(trainingData=True, numSamples=50):
    ret = {}
    for qu in queries.values():
        for ek, en in qu.items():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if len(ret) > numSamples:
                        return ret
                    m = max(en['vals'].values())
                    g = en['vals'].get(en['gold'], 0)
                    if g != m and g != 0:
                        ret[ek] = en
    return ret

In [38]:
queries_exp.check_params()

In [39]:
queries_exp.num_training_items = 250000# 250000

In [40]:
queries_exp.num_training_items = 50000
%time print queries_exp.compute_batch()

(0.19255790085869953, 0.19255790085869953)
CPU times: user 10min 33s, sys: 8.21 s, total: 10min 41s
Wall time: 1min 21s


In [309]:
def gg_test():
    self = queries_exp
    gg_f = theano.function(
            self.func_inputs,
            T.grad(self.loss_vec.sum(), [self.linear_features_dens_l.W]),
            updates=self.updates,
            on_unused_input='ignore',
    )
    return gg_f( self.current_documents,
            self.current_surface_link, self.current_surface_context, self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id, 
            self.current_target_words, self.current_target_body_words, self.current_feat_indicators,
            self.current_target_goal, self.current_learning_groups, self.current_boosted_groups,
        )

gg_res = gg_test()
gg_res

[array([[-0.01040162],
        [-0.10106535],
        [-0.12414555],
        ..., 
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]], dtype=float32)]

In [41]:
findWrongItems()

{u' [Citronella oil] can be burned in wick lamps outdoors .': {'boosted': 2.0,
  u'gold': u'Citronella oil',
  u'training': True,
  u'vals': {u'-NIL-': [(2.266223430633545, 0.2794010043144226), [87, 124]],
   u'Citronella oil': [(2.752781629562378, 0.7169269323348999),
    [88, 90, 93, 95, 104, 105, 111, 112, 116, 117, 118, 119]],
   u'Cooking oil': [(2.6866421699523926, 0.6919172406196594),
    [88,
     95,
     104,
     105,
     109,
     111,
     112,
     115,
     116,
     118,
     141,
     162,
     163,
     166,
     167,
     183,
     184,
     188,
     189,
     230,
     484]],
   u'Fuel oil': [(2.609722852706909, 0.616443932056427),
    [88,
     95,
     104,
     106,
     108,
     109,
     111,
     112,
     114,
     115,
     141,
     162,
     163,
     166,
     167,
     183,
     184,
     188,
     189,
     235,
     236]],
   u'List of Dynasty episodes': [(2.1522104740142822, 0.16753116250038147),
    [88,
     104,
     105,
     111,
     112,
   

In [ ]:
len(queries_exp.current_surface_target_counts)

In [ ]:
evalCurrentState(False, 500000)

In [ ]:
evalCurrentState(True, 500000)

In [ ]:
exp_results = []

true_num_samples = 500000 #queries_exp.num_training_items

for i in xrange(10):
    queries_exp.num_training_items = true_num_samples
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
    #if i % 2 == 1:
    exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
    queries_exp.num_training_items = 50000  # don't need that many samples to see how well it performs
    exp_results.append(('testing run', queries_exp.compute_batch(False)))
    exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [60]:
exp_results

[(0, (0.039299484917295457, 0.039299484917295457)),
 ('training state', (14649, 0.8748037408696839)),
 ('testing run', (0.040128257274907334, 0.040128257274907334)),
 ('testing state', (3230, 0.714860681114551)),
 (1, (0.016872953411064515, 0.016872953411064515)),
 ('training state', (14649, 0.9441600109222472)),
 ('testing run', (0.035271273664822814, 0.035271273664822814)),
 ('testing state', (3230, 0.7213622291021672)),
 (2, (0.011325508965034177, 0.011325508965034177)),
 ('training state', (14649, 0.9585637244863131)),
 ('testing run', (0.033288778484848743, 0.033288778484848743)),
 ('testing state', (3230, 0.7253869969040248)),
 (3, (0.0078287334997640904, 0.0078287334997640904)),
 ('training state', (14649, 0.9662092975629736)),
 ('testing run', (0.031593552965981202, 0.031593552965981202)),
 ('testing state', (3230, 0.7260061919504643)),
 (4, (0.0055831790501666874, 0.0055831790501666874)),
 ('training state', (14649, 0.9707147245545771)),
 ('testing run', (0.031277783450125045,

In [51]:
exp_results

[(0, (0.039404676521635318, 0.039404676521635318)),
 ('training state', (14649, 0.87439415659772)),
 ('testing run', (0.042006891875213524, 0.042006891875213524)),
 ('testing state', (3230, 0.7102167182662539)),
 (1, (0.017275828419484787, 0.017275828419484787)),
 ('training state', (14649, 0.9419755614717729)),
 ('testing run', (0.037033385882679834, 0.037033385882679834)),
 ('testing state', (3230, 0.7176470588235294)),
 (2, (0.011478287374442028, 0.011478287374442028)),
 ('training state', (14649, 0.9569936514437846)),
 ('testing run', (0.033139580750841051, 0.033139580750841051)),
 ('testing state', (3230, 0.721671826625387)),
 (3, (0.0083152518015077986, 0.0083152518015077986)),
 ('training state', (14649, 0.965458393064373)),
 ('testing run', (0.032309930766491531, 0.032309930766491531)),
 ('testing state', (3230, 0.7222910216718266)),
 (4, (0.0054266630257204362, 0.0054266630257204362)),
 ('training state', (14649, 0.9711925728718683)),
 ('testing run', (0.032920114213924316, 0.

In [49]:
exp_results

[(0, (0.040305165936308722, 0.040305165936308722)),
 ('training state', (14649, 0.8704348419687351)),
 ('testing run', (0.04278509938190684, 0.04278509938190684)),
 ('testing state', (3230, 0.7003095975232199)),
 (1, (0.018278701744695987, 0.018278701744695987)),
 ('training state', (14649, 0.93719707829886)),
 ('testing run', (0.038880461936085194, 0.038880461936085194)),
 ('testing state', (3230, 0.7164086687306501)),
 (2, (0.012382061340425484, 0.012382061340425484)),
 ('training state', (14649, 0.9537852413134003)),
 ('testing run', (0.035746649911481305, 0.035746649911481305)),
 ('testing state', (3230, 0.7256965944272445)),
 (3, (0.0089603640130947494, 0.0089603640130947494)),
 ('training state', (14649, 0.9634787357498805)),
 ('testing run', (0.034013545342567786, 0.034013545342567786)),
 ('testing state', (3230, 0.725077399380805)),
 (4, (0.0061412088385403765, 0.0061412088385403765)),
 ('training state', (14649, 0.968803331285412)),
 ('testing run', (0.033589602535108502, 0.03

In [46]:
exp_results

[(0, (0.041939613234384866, 0.041939613234384866)),
 ('training state', (14649, 0.8690012970168612)),
 ('testing run', (0.049184166765680958, 0.049184166765680958)),
 ('testing state', (3230, 0.701547987616099)),
 (1, (0.020761230615765468, 0.020761230615765468)),
 ('training state', (14649, 0.9283910164516349)),
 ('testing run', (0.043142837726674155, 0.043142837726674155)),
 ('testing state', (3230, 0.7114551083591332)),
 (2, (0.014823610463861938, 0.014823610463861938)),
 ('training state', (14649, 0.9460714041914123)),
 ('testing run', (0.042715882364325941, 0.042715882364325941)),
 ('testing state', (3230, 0.7151702786377709)),
 (3, (0.010964116493425923, 0.010964116493425923)),
 ('training state', (14649, 0.9568571233531299)),
 ('testing run', (0.04226961259422074, 0.04226961259422074)),
 ('testing state', (3230, 0.7198142414860681)),
 (4, (0.0075941112863683125, 0.0075941112863683125)),
 ('training state', (14649, 0.9664140896989556)),
 ('testing run', (0.041347561600518036, 0.0

In [54]:
evalCurrentStateF1(False, 50000)

(3247, 'Prec = 1366.0/1821 = 0.750137287205, Rec = 1366.0/2037 = 0.6705940108, F1 = 0.708138932089')


(3247,
 'Prec = 1366.0/1821 = 0.750137287205, Rec = 1366.0/2037 = 0.6705940108, F1 = 0.708138932089')

In [55]:
findWrongItems(False)

{u" [Baltimore] 's Eddie Murray cracked his 20th homer of the season": {'boosted': 10.0,
  u'gold': u'Baltimore Orioles',
  u'training': False,
  u'vals': {u'-NIL-': [(-1.2522202730178833, 0), [15, 50]],
   u'2011 Baltimore Ravens season': [(-1.491068720817566, 0),
    [23, 26, 29, 30, 32, 35, 48, 49, 78, 79, 80, 82, 84, 94, 105]],
   u'2012 Baltimore Ravens season': [(-2.134450674057007, 0),
    [23, 26, 29, 30, 32, 33, 35, 48, 49, 78, 79, 80, 82, 84, 103]],
   u'Baltimore': [(7.263938903808594, 0),
    [23, 24, 26, 27, 29, 30, 32, 33, 35, 77, 82, 111]],
   u'Baltimore (Parliament of Ireland constituency)': [(-1.5275434255599976,
     0),
    [23, 26, 27, 29, 32, 33, 35, 48, 78, 79, 84, 85, 93, 96, 98, 99]],
   u'Baltimore Clippers': [(-1.1133906841278076, 0),
    [26, 27, 32, 33, 35, 48, 78, 79, 82, 84, 93, 98, 112, 114, 116]],
   u'Baltimore County, Maryland': [(0.3490173816680908, 0),
    [23, 24, 26, 27, 29, 30, 32, 33, 35, 78, 79, 82, 83, 84, 98]],
   u'Baltimore Orioles': [(5.60

In [44]:
findWrongItems(True)

{u' [ABC] ': {'boosted': 10.0,
  u'gold': u'ABC (newspaper)',
  u'training': True,
  u'vals': {u'-NIL-': [(-3.6431639194488525, 0), [15, 50]],
   u'ABC': [(-1.5613656044006348, 0),
    [23, 26, 29, 32, 35, 48, 49, 82, 93, 94, 103, 111]],
   u'ABC (Australian TV channel)': [(-1.6718940734863281, 0),
    [23, 26, 29, 30, 32, 33, 35, 48, 49, 78, 79, 84, 85, 95, 98, 99]],
   u'ABC (band)': [(-2.2466375827789307, 0),
    [23, 26, 29, 30, 32, 33, 35, 48, 49, 78, 79, 84, 85, 97, 98, 99]],
   u'ABC (newspaper)': [(3.577418088912964, 0),
    [35, 48, 49, 78, 79, 84, 85, 90, 93, 94, 98, 99, 114, 115, 116, 117]],
   u'ABC Futebol Clube': [(-2.7322535514831543, 0),
    [23, 26, 29, 32, 35, 48, 49, 78, 79, 82, 84, 87, 93, 94, 98]],
   u'ABC Records': [(-1.416487216949463, 0),
    [23, 24, 26, 27, 29, 30, 32, 33, 35, 78, 79, 82, 84, 98, 101]],
   u'ABC Television': [(-2.535629987716675, 0),
    [23, 26, 29, 30, 32, 33, 35, 48, 49, 78, 79, 82, 84, 96, 98]],
   u'American Broadcasting Company': [(3.92

In [40]:
queries_exp.linear_features_dens_l.W.get_value(borrow=True)

array([[ 1.89938962],
       [ 2.37303758],
       [ 1.90854526],
       [ 2.06370664],
       [ 2.6816721 ],
       [ 2.70651293],
       [ 1.26042461],
       [ 0.9519667 ],
       [-0.0447967 ],
       [-0.29575172],
       [-0.72531062],
       [-1.77927506]], dtype=float32)

In [438]:
exp_results

[(0, (0.15075959757855004, 0.15075959757855004)),
 ('training state', (16838, 0.43704715524409077)),
 (0, (0.14573394275202456, 0.14573394275202456)),
 ('training state', (16838, 0.41661717543651267)),
 ('testing run', (0.13791946411895448, 0.13791946411895448)),
 ('testing state', (15226, 0.366938132142388)),
 (0, (0.14364645755698435, 0.14364645755698435)),
 ('training state', (16838, 0.4219028388169616)),
 (0, (0.14179708870844485, 0.14179708870844485)),
 ('training state', (16838, 0.42736667062596506)),
 ('testing run', (0.13767964049941461, 0.13767964049941461)),
 ('testing state', (3018, 0.3538767395626243)),
 (1, (0.14024924805706177, 0.14024924805706177)),
 ('training state', (16838, 0.4349091341014372)),
 ('testing run', (0.1369948137813807, 0.1369948137813807)),
 ('testing state', (3018, 0.35255135851557323)),
 (2, (0.13881489448158499, 0.13881489448158499)),
 ('training state', (16838, 0.4394821237676684)),
 ('testing run', (0.13580542270742596, 0.13580542270742596)),
 ('tes

In [428]:
exp_results

[(0, (0.15075959757855004, 0.15075959757855004)),
 ('training state', (16838, 0.43704715524409077)),
 (0, (0.14573394275202456, 0.14573394275202456)),
 ('training state', (16838, 0.41661717543651267)),
 ('testing run', (0.13791946411895448, 0.13791946411895448)),
 ('testing state', (15226, 0.366938132142388)),
 (0, (0.14364645755698435, 0.14364645755698435)),
 ('training state', (16838, 0.4219028388169616)),
 (0, (0.14179708870844485, 0.14179708870844485)),
 ('training state', (16838, 0.42736667062596506)),
 ('testing run', (0.13767964049941461, 0.13767964049941461)),
 ('testing state', (3018, 0.3538767395626243)),
 (1, (0.14024924805706177, 0.14024924805706177)),
 ('training state', (16838, 0.4349091341014372)),
 ('testing run', (0.1369948137813807, 0.1369948137813807)),
 ('testing state', (3018, 0.35255135851557323)),
 (2, (0.13881489448158499, 0.13881489448158499)),
 ('training state', (16838, 0.4394821237676684)),
 ('testing run', (0.13580542270742596, 0.13580542270742596)),
 ('tes

In [405]:
queries_exp.updates.keys()[-6].get_value(borrow=True).min()

0.0

In [420]:
findWrongItems()

{u' [Citronella oil] can be burned in wick lamps outdoors .': {'boosted': 10.0,
  u'gold': u'Citronella oil',
  u'training': True,
  u'vals': {u'-NIL-': [(-1.6257750988006592, 0), [87, 124]],
   u'Citronella oil': [(4.7099714279174805, 0),
    [88, 90, 93, 95, 104, 105, 111, 112, 116, 117, 118, 119]],
   u'Cooking oil': [(5.390409469604492, 0),
    [88,
     95,
     104,
     105,
     109,
     111,
     112,
     115,
     116,
     118,
     141,
     162,
     163,
     166,
     167,
     183,
     184,
     188,
     189,
     230,
     484]],
   u'Fuel oil': [(5.317875385284424, 0),
    [88,
     95,
     104,
     106,
     108,
     109,
     111,
     112,
     114,
     115,
     141,
     162,
     163,
     166,
     167,
     183,
     184,
     188,
     189,
     235,
     236]],
   u'List of Dynasty episodes': [(1.3453088998794556, 0),
    [88,
     104,
     105,
     111,
     112,
     116,
     117,
     118,
     119,
     141,
     183,
     186,
     187,
     

In [409]:
exp_results

[(0, (0.15075959757855004, 0.15075959757855004)),
 ('training state', (16838, 0.43704715524409077)),
 (0, (0.14573394275202456, 0.14573394275202456)),
 ('training state', (16838, 0.41661717543651267)),
 ('testing run', (0.13791946411895448, 0.13791946411895448)),
 ('testing state', (15226, 0.366938132142388))]

In [407]:
evalCurrentStateF1(False, 5000)

(320, 'Prec = 104.0/285 = 0.364912280702, Rec = 104.0/299 = 0.347826086957, F1 = 0.356164383562')


(320,
 'Prec = 104.0/285 = 0.364912280702, Rec = 104.0/299 = 0.347826086957, F1 = 0.356164383562')

In [463]:
(queries_exp.linear_features_dens_l.W.get_value(borrow=True) > .00019).mean()

0.010877472546506391

In [459]:
queries_exp.linear_features_dens_l.W.get_value(borrow=True)[0:40]

array([[  2.60732579e+00],
       [  3.53522539e+00],
       [  3.34248996e+00],
       [ -7.38936709e-04],
       [ -3.67235276e-03],
       [ -4.89456579e-02],
       [  2.66805124e-02],
       [ -2.33040620e-02],
       [  1.67117640e-02],
       [  1.72123611e-02],
       [  5.56132710e-03],
       [  3.00122201e-02],
       [  8.88470002e-03],
       [ -6.31554276e-02],
       [ -2.90955063e-02],
       [  2.66024489e-02],
       [ -5.00463136e-03],
       [ -1.16568711e-02],
       [  8.45910632e-04],
       [ -9.44055896e-03],
       [ -1.45682422e-02],
       [  6.52521523e-03],
       [ -3.08596827e-02],
       [  1.49343954e-03],
       [  8.07015225e-04],
       [  8.63364339e-03],
       [  2.65006963e-02],
       [  8.65139533e-04],
       [ -4.27377596e-03],
       [ -3.66638675e-02],
       [ -9.32722446e-03],
       [ -1.38056064e-02],
       [ -3.60027850e-02],
       [  2.07976084e-02],
       [  8.91434681e-03],
       [ -2.56620068e-03],
       [ -2.41660401e-02],
 

In [456]:
(abs(queries_exp.linear_features_dens_l.W.get_value(borrow=True))**2).sum()

140.62112

In [447]:
(queries_exp.linear_features_dens_l.W.get_value(borrow=True)[3:900] > 1).sum()

8

In [156]:
queries_exp.linear_features_dens_l.W.get_value(borrow=True)[0] = 0

In [157]:
queries_exp.num_training_items = 50000

In [158]:
queries_exp.compute_batch(False)

(0.043889810960610752, 0.043889810960610752)

In [200]:
evalCurrentState(False, queries_exp.num_training_items)

(22944, 0.8301516736401674)


(22944, 0.8301516736401674)

In [ ]:
#queries_exp.num_training_items = 50000
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [198]:
exp_results

[(0, (0.042415848140430139, 0.042415848140430139)),
 ('training state', (100813, 0.8506145040818148)),
 ('testing run', (0.040505639483841581, 0.040505639483841581)),
 ('testing state', (22944, 0.8260983263598326)),
 (1, (0.034083649006694042, 0.034083649006694042)),
 ('training state', (100813, 0.8676063602908355)),
 ('testing run', (0.039983913868662346, 0.039983913868662346)),
 ('testing state', (22944, 0.827928870292887)),
 (2, (0.032209361035501115, 0.032209361035501115)),
 ('training state', (100813, 0.8710086992748951)),
 ('testing run', (0.039615782901552529, 0.039615782901552529)),
 ('testing state', (22944, 0.8300209205020921)),
 (3, (0.03082796823178836, 0.03082796823178836)),
 ('training state', (100813, 0.874490393104064)),
 ('testing run', (0.039272816533898185, 0.039272816533898185)),
 ('testing state', (22944, 0.8305003486750349)),
 (4, (0.029458737707327584, 0.029458737707327584)),
 ('training state', (100813, 0.8783886998700564)),
 ('testing run', (0.03937902614214509

In [ ]:
exp_results

In [ ]:
surface_counts['The Silent World of Nicholas Quinn'.lower()]

In [ ]:
surface_counts['canada']

In [ ]:
surface_counts['urban district']

In [ ]:
findWrongItems(True, 100)

In [ ]:

exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [ ]:
exp_results

In [ ]:
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [ ]:
for i in xrange(4):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

In [ ]:
queries_exp.all_params

In [ ]:
queries_exp.all_params[-2].get_value(borrow=True)

In [ ]:
queries_exp.all_params[-2:][0].get_value(borrow=True)

In [ ]:
saved_lin_layer

In [ ]:
findWrongItems()

In [ ]:
findWrongItems(False)

In [ ]:
surface_counts['The Silent World of Nicholas Quinn'.lower()]

In [ ]:
%time for i in range(5): print queries_exp.compute_batch()

In [ ]:
queries.values()[0]

In [ ]:
queries_exp.all_params

In [ ]:
queries_exp.all_params[-2].get_value(borrow=True)

In [ ]:
queries_exp.all_params[-2].get_value(borrow=True)[0] = 20.0

In [ ]:
len(queries_exp.page_content.values()[1])